# 실습 세션 1: LLM 및 프롬프트 엔지니어링 활용

이번 세션에서는 Azure AI Foundry에서 배포한 LLM을 활용하여 Chat Playground와 Jupyter Notebook에서 다양한 프롬프트 엔지니어링 기법을 실습합니다.

# LLM(대형 언어 모델) 개요 및 아키텍처

LLM(Large Language Model)은 대규모 텍스트 데이터를 기반으로 사전 학습된 인공지능 언어 모델로, 자연어 이해와 생성, 요약, 번역, 질의응답 등 다양한 작업에 활용됩니다. 대표적인 LLM으로는 OpenAI의 GPT 시리즈(GPT-3, GPT-4, GPT-4o), Google의 Gemini(구 PaLM), Meta의 Llama, Anthropic의 Claude, Microsoft Azure OpenAI 등이 있습니다.

- **주요 특징**
  - 방대한 데이터와 파라미터(수십억~수천억 개)를 활용해 인간 수준의 언어 이해 및 생성 능력을 보유
  - 프롬프트(지시문)만으로 다양한 태스크를 수행할 수 있는 범용성
  - API 형태로 손쉽게 활용 가능 (Azure OpenAI, OpenAI API 등)
  - 프롬프트 엔지니어링, 파인튜닝 등 다양한 활용법 지원

### LLM의 토큰, 인코딩/디코딩, 확률 개념
- **토큰(Token)**: LLM은 텍스트를 단어, 부분 단어, 기호 등으로 쪼갠 '토큰' 단위로 처리합니다. 예를 들어, "Azure OpenAI"는 2~3개의 토큰으로 분할될 수 있습니다. 모델의 입력/출력 길이, 비용, 성능 등은 토큰 수에 따라 결정됩니다.
- **인코딩(Encoding) / 디코딩(Decoding)**: 인코딩은 입력 텍스트를 토큰 ID(숫자 시퀀스)로 변환하는 과정이고, 디코딩은 모델이 생성한 토큰 ID를 다시 사람이 읽을 수 있는 텍스트로 변환하는 과정입니다. 다양한 인코딩 방식(BPE, WordPiece 등)이 사용됩니다.
- **확률(Probability)**: LLM은 다음에 올 토큰을 예측할 때 각 후보 토큰에 확률을 할당합니다. temperature, top-p 등 파라미터는 이 확률 분포를 조절해 답변의 다양성과 창의성을 결정합니다. 확률이 높은 토큰은 더 자주 선택되고, 낮은 temperature나 top-p는 더 결정적인(예측 가능한) 답변을 만듭니다.

이러한 개념을 이해하면 LLM의 동작 원리와 파라미터 조정이 결과에 미치는 영향을 더 잘 파악할 수 있습니다.

아래는 LLM 내부의 기본 아키텍처 예시입니다.

```
[입력 텍스트]
   |
   v
[토크나이저 (Tokenizer): 텍스트 → 토큰]
   |
   v
[임베딩 레이어 (Embedding Layer): 토큰 → 벡터]
   |
   v
[트랜스포머 블록 (Transformer Blocks): 인코더/디코더, 셀프 어텐션 등]
   |
   v
[로짓/확률 분포 계산 (Logits/Probability Distribution)]
   |
   v
[디코더: 확률에 따라 다음 토큰 샘플링]
   |
   v
[출력 토큰 → 텍스트 변환 (Detokenizer)]
   |
   v
[최종 응답 텍스트]
```

- 입력 텍스트는 토크나이저를 통해 토큰으로 변환되고, 임베딩 레이어에서 벡터로 변환됩니다.
- 트랜스포머 블록에서 문맥을 이해하고, 다음에 올 토큰의 확률 분포를 계산합니다.
- 디코더는 temperature, top-p 등 파라미터에 따라 다음 토큰을 샘플링합니다.
- 최종적으로 토큰 시퀀스가 텍스트로 변환되어 응답이 생성됩니다.

---

## 1. 패키지 설치 및 Azure OpenAI 연결 설정

실습에 필요한 패키지를 설치하고, Azure OpenAI 서비스에 연결하기 위한 환경설정을 진행합니다.

- requirements.txt 파일을 활용해 필요한 라이브러리를 설치합니다.
- .env 파일에는 Azure OpenAI 서비스의 엔드포인트, API 키, 배포 이름 등 환경변수가 이미 설정되어 있어야 합니다. (자세한 설정 방법은 README.md 참고)
- 아래 코드를 실행해 환경변수 로드 및 Azure OpenAI 클라이언트 연결이 정상적으로 되는지 확인하세요.

### 패키지 설치 및 커널 재시작 안내

- 아래 셀을 실행하여 필요한 라이브러리를 설치하세요.
- 설치가 완료된 후에는 **커널을 반드시 재시작**해야 모든 패키지가 정상적으로 인식됩니다.
- 커널 재시작 방법: 메뉴에서 'Runtime' > 'Restart Kernel' 또는 상단의 재시작 아이콘 클릭

In [16]:
# 필요한 라이브러리 설치
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

# .env 파일에서 환경 변수 로드 (없는 경우 직접 입력)
load_dotenv()

# Azure OpenAI 설정
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

# AzureOpenAI 클라이언트 생성
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version=api_version
)

# 모델 배포 이름
# gpt-4o mini 배포 시 아래와 같이 이름을 지정할 수 있습니다.
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME") # 배포된 모델 이름

## 2. Azure OpenAI LLM 모델 연결 및 응답 테스트

이 단계에서는 앞서 배포한 gpt-4o mini 등 Azure OpenAI LLM 모델이 정상적으로 연결되어 있는지 확인하고, 실제로 간단한 메시지를 보내 응답을 받아봅니다.

- 환경 변수(.env)에 입력한 정보로 Azure OpenAI 서비스에 연결합니다.
- 배포된 모델 이름(deployment_name)을 사용해 API 요청을 보냅니다.
- 연결 및 응답 테스트를 통해 모델이 정상적으로 동작하는지 확인할 수 있습니다.
- 만약 오류가 발생하면, 환경 변수 값과 Azure Portal의 배포 상태를 다시 확인하세요.

In [2]:
# 배포된 모델 목록 확인
try:
    # 참고: 현재 버전의 AzureOpenAI 클라이언트에서는 
    # 모든 배포 목록을 직접 확인하는 기능이 제한적일 수 있습니다.
    # 아래 코드는 예시일 뿐이며, 실제 환경에 따라 다를 수 있습니다.
    print("배포된 모델 정보:")
    print(f"- 현재 사용 중인 모델: {deployment_name}")
    print("Azure Portal에서 배포된 모든 모델을 확인할 수 있습니다.")

    # 배포된 모델 테스트 (간단한 테스트)
    test_response = client.chat.completions.create(
        model=deployment_name,
        messages=[{"role": "user", "content": "안녕하세요"}],
        max_tokens=10
    )
    print(f"\n모델 응답 테스트 성공: {test_response.choices[0].message.content}")
except Exception as e:
    print(f"모델 확인 중 오류 발생: {e}")
    print("Azure Portal에서 직접 배포 상태를 확인해주세요.")

배포된 모델 정보:
- 현재 사용 중인 모델: gpt-4o-mini
Azure Portal에서 배포된 모든 모델을 확인할 수 있습니다.

모델 응답 테스트 성공: 안녕하세요! 어떻게 도와드릴까요?


## 3. 기본 프롬프트 엔지니어링

이 단계에서는 LLM의 응답 품질을 높이기 위한 다양한 프롬프트 엔지니어링 기법을 실습합니다.

- 명확한 지시사항, 역할 부여, 단계별 사고 유도, Few-Shot 학습 등 다양한 프롬프트 작성법을 실습합니다.
- 각 기법을 적용한 프롬프트와 적용하지 않은 프롬프트의 차이를 비교해볼 수 있습니다.
- 실습을 통해 실제 업무나 프로젝트에서 활용할 수 있는 프롬프트 설계 역량을 키울 수 있습니다.
- 아래 예시 코드를 직접 수정·실행하며 다양한 결과를 실험해보세요.

In [3]:

def get_completion(prompt, model=deployment_name):
    """지정된 프롬프트와 모델을 사용하여 Azure OpenAI API에 요청을 보내고 응답을 반환합니다."""
    # 메시지 구성
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    # 채팅 완성 요청
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=4096
    )
    
    # 응답에서 메시지 내용 추출
    return response.choices[0].message.content

# 간단한 프롬프트 테스트
simple_prompt = "Azure란 무엇인가요?"
print(get_completion(simple_prompt))

Azure는 Microsoft에서 제공하는 클라우드 컴퓨팅 플랫폼으로, 다양한 클라우드 서비스와 솔루션을 제공합니다. Azure는 기업과 개발자들이 애플리케이션을 구축, 배포 및 관리할 수 있도록 지원하며, 이를 통해 다양한 IT 리소스를 유연하게 사용할 수 있습니다. Azure의 주요 기능과 서비스는 다음과 같습니다:

1. **IaaS(인프라스트럭처 서비스)**: 서버, 스토리지, 네트워크 등의 인프라를 클라우드에서 제공하여 사용자가 필요에 따라 리소스를 관리할 수 있습니다.

2. **PaaS(플랫폼 서비스)**: 개발자들이 애플리케이션을 개발하고 배포할 수 있는 환경을 제공합니다. 데이터베이스, 개발 도구, 애플리케이션 호스팅 등의 기능이 포함됩니다.

3. **SaaS(소프트웨어 서비스)**: 클라우드 기반의 소프트웨어 애플리케이션을 제공하여 사용자가 설치나 관리 없이 바로 사용할 수 있도록 합니다.

4. **데이터베이스 서비스**: Azure SQL Database, Cosmos DB 등 다양한 데이터베이스 서비스를 제공하여 데이터 저장 및 관리가 용이합니다.

5. **AI 및 머신러닝**: Azure Machine Learning과 같은 서비스를 통해 AI 모델을 구축하고 배포할 수 있습니다.

6. **IoT(사물인터넷)**: IoT Hub와 같은 서비스를 통해 IoT 기기를 연결하고 관리할 수 있습니다.

7. **보안 및 관리**: Azure는 다양한 보안 기능과 관리 도구를 제공하여 데이터 보호 및 리소스 관리를 지원합니다.

Azure는 전 세계 여러 지역에 데이터 센터를 운영하고 있어, 사용자들은 원하는 지역에서 서비스를 선택할 수 있으며, 높은 가용성과 확장성을 제공합니다. 기업들이 클라우드로 전환하고 디지털 혁신을 이루는 데 중요한 역할을 하고 있습니다.


### 3.1 명확한 지시사항 제공하기

LLM에게 명확한 지시사항을 제공하면 더 좋은 결과를 얻을 수 있습니다.

In [4]:
# 명확하지 않은 지시사항
unclear_prompt = "클라우드 컴퓨팅에 대해 알려줘."
print("\n=== 명확하지 않은 지시사항 ===\n")
print(get_completion(unclear_prompt))

# 명확한 지시사항
clear_prompt = """클라우드 컴퓨팅에 대해 다음 형식으로 설명해줘:
1. 정의 (50단어 이내)
2. 주요 이점 (5가지 나열)
3. 주요 서비스 제공업체 (3개)
4. Azure의 주요 서비스 (5가지)
"""
print("\n=== 명확한 지시사항 ===\n")
print(get_completion(clear_prompt))


=== 명확하지 않은 지시사항 ===

클라우드 컴퓨팅(Cloud Computing)은 인터넷을 통해 데이터 저장, 관리, 처리, 분석 등을 제공하는 서비스 모델입니다. 사용자는 물리적인 서버나 인프라를 직접 구매하거나 유지 관리할 필요 없이, 필요에 따라 컴퓨팅 리소스를 사용할 수 있습니다. 클라우드 컴퓨팅의 주요 특징과 장점은 다음과 같습니다.

### 주요 특징

1. **온디맨드 셀프 서비스(On-Demand Self-Service)**:
   사용자는 필요할 때 언제든지 클라우드 리소스를 요청하고 사용할 수 있습니다.

2. **광범위한 네트워크 접근성(Broad Network Access)**:
   클라우드 서비스는 인터넷이 연결된 장치에서 접근할 수 있습니다. 이는 PC, 노트북, 스마트폰 등 다양한 디바이스를 포함합니다.

3. **리소스 풀링(Resource Pooling)**:
   여러 사용자의 요구를 충족하기 위해 클라우드 서비스 제공자는 리소스를 여러 사용자 간에 공유하고 동적으로 할당합니다.

4. **빠른 탄력성(Rapid Elasticity)**:
   사용자는 필요에 따라 리소스를 신속하게 확장하거나 축소할 수 있습니다. 이는 클라우드 환경의 중요한 장점입니다.

5. **측정된 서비스(Metered Service)**:
   클라우드 서비스는 사용량에 따라 비용이 청구됩니다. 사용자는 자신이 사용한 만큼만 비용을 지불하게 됩니다.

### 클라우드 서비스 모델

클라우드 컴퓨팅은 일반적으로 세 가지 서비스 모델로 나눌 수 있습니다:

1. **인프라스트럭처 서비스(IaaS, Infrastructure as a Service)**:
   서버, 저장소, 네트워크와 같은 기본적인 IT 인프라를 제공합니다. 예: Amazon EC2, Google Compute Engine.

2. **플랫폼 서비스(PaaS, Platform as a Service)**:
   애플리케이션 개발 및 배포를 위한 플랫폼을 제공합니다. 개발자는 인프라

### 3.2. 역할 부여하기 (Role Prompting)

LLM에게 특정 역할을 부여하면 해당 관점에서 응답을 생성할 수 있습니다.

In [5]:
# 역할 없는 프롬프트
no_role_prompt = "Azure Kubernetes Service(AKS)의 장점을 설명해줘."
print("\n=== 역할 없는 프롬프트 ===\n")
print(get_completion(no_role_prompt))

# 역할 부여 프롬프트
role_prompt = """당신은 5년 경력의 클라우드 아키텍트입니다. 
기업의 IT 관리자에게 Azure Kubernetes Service(AKS)를 도입해야 하는 이유와 장점을 
기술적 측면과 비즈니스 측면에서 설득력 있게 설명해주세요."""
print("\n=== 역할 부여 프롬프트 ===\n")
print(get_completion(role_prompt))


=== 역할 없는 프롬프트 ===

Azure Kubernetes Service (AKS)는 Microsoft Azure에서 제공하는 관리형 Kubernetes 서비스로, 여러 가지 장점을 가지고 있습니다. 다음은 AKS의 주요 장점입니다.

1. **간편한 배포 및 관리**:
   - AKS는 Kubernetes 클러스터의 배포와 관리를 간소화합니다. Azure 포털, Azure CLI, ARM 템플릿 등을 통해 손쉽게 클러스터를 생성하고 관리할 수 있습니다.

2. **자동화된 업데이트 및 유지 관리**:
   - AKS는 Kubernetes 버전 업데이트 및 패치 관리를 자동으로 처리하여 사용자가 클러스터 유지 관리에 소요되는 시간을 줄일 수 있습니다.

3. **스케일링**:
   - AKS는 수요에 따라 클러스터를 쉽게 스케일 업 또는 스케일 다운할 수 있는 기능을 제공합니다. 자동 스케일링 기능을 통해 필요에 따라 노드 수를 조정할 수 있습니다.

4. **보안**:
   - Azure의 보안 모델을 활용할 수 있어, 네트워크 보안, IAM(Identity and Access Management), Azure Active Directory 통합 등을 통해 클러스터의 보안을 강화할 수 있습니다.

5. **통합된 모니터링 및 로깅**:
   - Azure Monitor와 Azure Log Analytics와 통합되어 클러스터의 성능과 상태를 모니터링하고, 로그 데이터를 수집하여 분석할 수 있습니다.

6. **서버리스 애플리케이션 지원**:
   - Azure Functions와 같은 서버리스 컴퓨팅 서비스를 AKS와 통합하여 이벤트 기반 아키텍처를 쉽게 구현할 수 있습니다.

7. **다양한 개발 언어 및 프레임워크 지원**:
   - AKS는 다양한 언어 및 프레임워크를 지원하며, 컨테이너화된 애플리케이션을 쉽게 배포할 수 있는 환경을 제공합니다.

8. **DevOps 통합**:
   - Azure DevOps와의 통합을 통해 CI/CD 파이프

### 3.3. 단계별 사고 유도 (Chain-of-Thought)

복잡한 문제를 해결할 때는 LLM에게 단계별로 생각하도록 유도하는 것이 효과적입니다.

In [6]:
# 직접적인 질문
direct_prompt = "Azure에서 3-tier 웹 애플리케이션을 구축하는 방법을 알려줘."
print("\n=== 직접적인 질문 ===\n")
print(get_completion(direct_prompt))

# 단계별 사고 유도 프롬프트
cot_prompt = """Azure에서 3-tier 웹 애플리케이션을 구축하려고 합니다. 
다음 단계에 따라 설계 방법을 설명해주세요:

1. 먼저, 각 계층(프레젠테이션, 비즈니스 로직, 데이터)에 적합한 Azure 서비스를 선택하세요.
2. 다음으로, 이러한 서비스들이 어떻게 연결되는지 설명하세요.
3. 그 다음, 보안 측면에서 고려해야 할 사항을 설명하세요.
4. 마지막으로, 확장성을 위한 설계 방법을 제안하세요.

각 단계별로 구분하여 자세히 설명해주세요."""
print("\n=== 단계별 사고 유도 프롬프트 ===\n")
print(get_completion(cot_prompt))


=== 직접적인 질문 ===

Azure에서 3-tier 웹 애플리케이션을 구축하는 방법은 다음과 같은 단계로 나눌 수 있습니다. 3-tier 아키텍처는 일반적으로 프레젠테이션 계층, 애플리케이션 계층, 데이터 계층으로 구성됩니다.

### 1. 아키텍처 설계

- **프레젠테이션 계층**: 사용자 인터페이스를 담당하는 계층. 일반적으로 웹 애플리케이션의 클라이언트 부분입니다.
- **애플리케이션 계층**: 비즈니스 로직을 처리하는 계층. API 서버 또는 애플리케이션 서버가 여기에 해당합니다.
- **데이터 계층**: 데이터베이스 및 데이터 저장소. 데이터의 저장, 검색, 관리 등을 담당합니다.

### 2. Azure 서비스 선택

- **프레젠테이션 계층**: Azure App Service를 사용하여 웹 애플리케이션을 호스팅할 수 있습니다.
- **애플리케이션 계층**: Azure Functions 또는 Azure App Service의 Web API를 사용하여 비즈니스 로직을 구현할 수 있습니다.
- **데이터 계층**: Azure SQL Database, Azure Cosmos DB, Azure Blob Storage 등 다양한 데이터 저장소 중에서 선택할 수 있습니다.

### 3. Azure 리소스 생성

1. **Azure Portal에 로그인**합니다.
2. **리소스 그룹**을 생성하거나 기존 리소스 그룹을 선택합니다.
3. 프레젠테이션 계층을 위한 **Azure App Service**를 생성합니다.
4. 애플리케이션 계층을 위한 **Azure Functions** 또는 **Azure App Service**를 생성합니다.
5. 데이터 계층을 위한 **Azure SQL Database** 또는 필요한 데이터 저장소를 생성합니다.

### 4. 애플리케이션 개발

- **프레젠테이션 계층**: HTML, CSS, JavaScript(예: React, Angular, Vue.js 등)를 사용하여 클라이언트 애플리케이션을 개발합니다.
- **

### 3.4. Few-Shot 학습

몇 가지 예시를 제공하여 LLM이 패턴을 학습하도록 유도할 수 있습니다.

In [7]:
# Few-shot 프롬프트
few_shot_prompt = """다음은 클라우드 서비스에 대한 간결한 설명입니다:

서비스: Azure Virtual Machine
설명: 다양한 크기와 운영체제를 지원하는 확장 가능한 컴퓨팅 리소스로, IaaS 솔루션을 제공합니다.

서비스: AWS Lambda
설명: 서버리스 컴퓨팅 서비스로, 코드 실행만 관리하며 인프라 관리가 필요 없는 이벤트 기반 애플리케이션을 구축할 수 있습니다.

서비스: Google Cloud Storage
설명: 확장 가능하고 내구성 있는 객체 스토리지 솔루션으로, 정적 콘텐츠 호스팅부터 빅데이터 분석까지 다양한 용도로 활용됩니다.

위의 형식을 따라 아래 서비스에 대한 설명을 작성해주세요:

서비스: Azure Kubernetes Service (AKS)
설명:

서비스: Azure Functions
설명:

서비스: Azure Cosmos DB
설명:
"""
print(get_completion(few_shot_prompt))

서비스: Azure Kubernetes Service (AKS)  
설명: Kubernetes를 기반으로 한 컨테이너 오케스트레이션 서비스로, 애플리케이션을 쉽게 배포, 관리 및 확장할 수 있도록 지원합니다.

서비스: Azure Functions  
설명: 서버리스 컴퓨팅 서비스로, 이벤트 기반으로 코드를 실행할 수 있으며, 인프라를 신경 쓰지 않고 애플리케이션 로직에 집중할 수 있습니다.

서비스: Azure Cosmos DB  
설명: 글로벌 분산형 NoSQL 데이터베이스 서비스로, 다양한 데이터 모델을 지원하며, 자동 스케일링과 낮은 지연시간을 제공하여 애플리케이션의 성능을 극대화합니다.


## 4. 고급 프롬프트 엔지니어링 기법

이 단계에서는 LLM을 더욱 효과적으로 활용하기 위한 고급 프롬프트 엔지니어링 기법을 실습합니다.

- 시스템 메시지, 다중 단계 지시사항, 출력 형식 지정 등 실제 서비스 개발에 활용할 수 있는 고급 프롬프트 작성법을 다룹니다.
- 각 기법을 통해 LLM의 응답 품질과 일관성을 높이고, 원하는 형태의 결과를 얻는 방법을 익힐 수 있습니다.
- 실습 코드를 직접 수정·실행하며 다양한 고급 프롬프트 전략을 실험해보세요.
- 실제 프로젝트나 서비스에 적용할 수 있는 실전 노하우를 쌓는 데 도움이 됩니다.

### 4.1. 시스템 메시지 활용하기

시스템 메시지를 사용하여 LLM의 전반적인 행동을 정의할 수 있습니다.

In [8]:
def get_completion_with_system(system_message, user_message, model=deployment_name):
    """시스템 메시지와 사용자 메시지를 사용하여 Azure OpenAI API에 요청을 보내고 응답을 반환합니다."""
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.7,
        max_tokens=800
    )
    return response.choices[0].message.content

# 기술 전문가 시스템 메시지
tech_system_message = """당신은 Microsoft Azure 전문가입니다. 항상 기술적으로 정확한 정보를 제공하며, 
복잡한 개념도 초보자가 이해할 수 있도록 쉽게 설명합니다. 가능한 경우 코드 예제나 구체적인 단계를 포함해주세요."""

user_question = "Azure Functions와 Azure Logic Apps의 차이점은 무엇인가요?"

print("=== 시스템 메시지 활용 ===\n")
print(get_completion_with_system(tech_system_message, user_question))

=== 시스템 메시지 활용 ===

Azure Functions와 Azure Logic Apps는 모두 Microsoft Azure에서 제공하는 서버리스 컴퓨팅 서비스이지만, 그 목적과 사용 방식이 다릅니다. 아래에 두 서비스의 주요 차이점을 설명하겠습니다.

### Azure Functions

1. **개념**: Azure Functions는 이벤트 기반의 서버리스 컴퓨팅 서비스로, 특정 이벤트가 발생했을 때 자동으로 코드를 실행할 수 있습니다. 예를 들어, HTTP 요청, 파일 업로드, 데이터베이스 변경 등 다양한 이벤트에 반응하여 함수를 실행할 수 있습니다.

2. **사용 사례**:
   - API 엔드포인트 제공
   - 데이터 처리 (예: 파일 변환, 데이터 정리)
   - 주기적으로 작업 실행 (예: 크론 작업)

3. **개발 방식**: Azure Functions는 주로 코드로 작성됩니다. 다양한 프로그래밍 언어(예: C#, JavaScript, Python 등)를 지원하며, 개발자가 직접 로직을 구현해야 합니다.

4. **유연성**: 개발자가 코드를 직접 작성하므로 복잡한 로직 구현이 가능합니다.

5. **비용 모델**: 사용한 만큼만 비용을 지불하는 소비 기반 모델입니다.

### Azure Logic Apps

1. **개념**: Azure Logic Apps는 다양한 서비스를 연결하여 자동화된 워크플로우를 생성할 수 있는 서비스입니다. 주로 비즈니스 프로세스를 자동화하는 데 사용되며, 코드 없이 시각적으로 워크플로우를 디자인할 수 있습니다.

2. **사용 사례**:
   - 여러 서비스 간 데이터 통합
   - 알림 및 승인 프로세스 자동화
   - 외부 API와의 통합

3. **개발 방식**: Logic Apps는 GUI 기반의 디자인 툴을 사용하여 워크플로우를 만들 수 있습니다. 사용자는 다양한 트리거와 액션을 선택하여 시각적으로 연결할 수 있습니다.

4. **유연성**: 주로 사전 정의된 트리거와 액션을 사용하므로, 복잡

### 4.2. 다중 단계 지시사항 (Multi-Step Instructions)

복잡한 작업을 여러 단계로 나누어 수행하도록 지시할 수 있습니다.

In [9]:
multi_step_prompt = """Azure Web App을 사용하여 간단한 웹 애플리케이션을 배포하는 과정을 알려주세요.
다음 순서로 설명해주세요:

1. 필요한 사전 요구 사항 나열
2. Azure Portal에서 Web App 리소스 생성 단계
3. VS Code를 사용하여 로컬에서 웹 앱 개발하기
4. Azure에 웹 앱 배포하기
5. 배포 후 모니터링 및 문제 해결 방법

각 단계마다 구체적인 명령어나 설정 옵션을 포함해주세요."""

print("=== 다중 단계 지시사항 ===\n")
print(get_completion(multi_step_prompt))

=== 다중 단계 지시사항 ===

Azure Web App을 사용하여 간단한 웹 애플리케이션을 배포하는 과정은 다음과 같습니다.

### 1. 필요한 사전 요구 사항 나열
- **Azure 계정**: Azure Portal에 로그인할 수 있는 Microsoft Azure 계정이 필요합니다. 계정이 없다면 [Azure 무료 계정](https://azure.microsoft.com/free/)을 생성할 수 있습니다.
- **Visual Studio Code**: VS Code가 설치되어 있어야 합니다. [VS Code 다운로드](https://code.visualstudio.com/) 링크에서 설치할 수 있습니다.
- **Azure CLI**: Azure CLI를 설치합니다. 설치 방법은 [Azure CLI 설치 가이드](https://docs.microsoft.com/cli/azure/install-azure-cli)에서 확인할 수 있습니다.
- **Node.js (선택적)**: Node.js를 사용하는 경우 [Node.js 다운로드](https://nodejs.org/)에서 설치합니다.
- **Git**: 버전 관리와 배포를 위해 Git이 필요합니다. [Git 다운로드](https://git-scm.com/)에서 설치합니다.

### 2. Azure Portal에서 Web App 리소스 생성 단계
1. Azure Portal에 로그인합니다.
2. 왼쪽 메뉴에서 "리소스 만들기"를 클릭합니다.
3. "웹" 카테고리에서 "Web App"을 선택합니다.
4. Web App의 기본 설정을 입력합니다:
   - **구독**: 사용할 구독 선택
   - **리소스 그룹**: 새 리소스 그룹 생성 또는 기존 리소스 그룹 선택
   - **앱 이름**: 고유한 웹 앱 이름 입력 (예: `myuniqueappname`)
   - **배포 방법**: "코드" 선택
   - **런타임 스택**: 사용할 런타임 스택 선택 (예: .NET, Node.js, Python 등)
   

### 4.3. 출력 형식 지정 (Output Formatting)

LLM의 응답을 특정 형식으로 받을 수 있습니다.

In [10]:
format_prompt = """Azure의 주요 AI 서비스 5가지를 JSON 형식으로 제공해주세요. 각 서비스에 대해 다음 정보를 포함해주세요:
- 서비스 이름
- 간단한 설명
- 주요 기능 (3가지)
- 일반적인 사용 사례 (2가지)
- 관련 Azure 서비스 (2가지)

응답은 다음과 같은 JSON 형식이어야 합니다:
{
  "ai_services": [
    {
      "name": "서비스 이름",
      "description": "설명",
      "key_features": ["기능1", "기능2", "기능3"],
      "use_cases": ["사용 사례1", "사용 사례2"],
      "related_services": ["관련 서비스1", "관련 서비스2"]
    },
    ...
  ]
}"""

json_response = get_completion(format_prompt)
print("=== 출력 형식 지정 (JSON) ===\n")
print(json_response)

# JSON 응답을 파싱하여 구조화된 데이터로 사용 가능
try:
    parsed_response = json.loads(json_response)
    print("\n=== 첫 번째 서비스 정보 ===\n")
    print(f"서비스 이름: {parsed_response['ai_services'][0]['name']}")
    print(f"설명: {parsed_response['ai_services'][0]['description']}")
    print("주요 기능:")
    for feature in parsed_response['ai_services'][0]['key_features']:
        print(f"- {feature}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 오류: {e}")

=== 출력 형식 지정 (JSON) ===

```json
{
  "ai_services": [
    {
      "name": "Azure Cognitive Services",
      "description": "Azure Cognitive Services는 자연어 처리, 컴퓨터 비전, 음성 인식 등 다양한 AI 기능을 제공하는 API 모음입니다.",
      "key_features": ["감정 분석", "객체 인식", "음성 합성"],
      "use_cases": ["고객 서비스 자동화", "콘텐츠 필터링 및 분류"],
      "related_services": ["Azure Machine Learning", "Azure Bot Services"]
    },
    {
      "name": "Azure Machine Learning",
      "description": "Azure Machine Learning은 데이터 과학자와 개발자가 모델을 구축, 학습 및 배포할 수 있도록 돕는 클라우드 기반 플랫폼입니다.",
      "key_features": ["자동화된 머신러닝", "데이터 준비 도구", "모델 관리"],
      "use_cases": ["예측 분석", "맞춤형 추천 시스템"],
      "related_services": ["Azure Databricks", "Azure Synapse Analytics"]
    },
    {
      "name": "Azure Bot Services",
      "description": "Azure Bot Services는 다양한 플랫폼에서 대화형 봇을 쉽게 구축하고 배포할 수 있도록 지원하는 서비스입니다.",
      "key_features": ["다양한 채널 통합", "애널리틱스 기능", "인공지능 지원"],
      "use_cases": ["고객 지원 챗봇", "정보 제공 봇"],
      "related_services": ["Azure Cogniti

## 5. 실전 프롬프트 엔지니어링

이 단계에서는 실제 업무나 프로젝트에서 바로 활용할 수 있는 프롬프트 엔지니어링 예제를 다룹니다.

- 실전 시나리오(아키텍처 설계, CLI 명령어 생성 등)에 맞는 프롬프트 작성법을 실습합니다.
- LLM을 통해 복잡한 요구사항을 효과적으로 해결하는 방법을 익힐 수 있습니다.
- 다양한 실전 예제를 직접 수정·실행하며, 자신만의 프롬프트 설계 역량을 강화해보세요.
- 실제 현업에서 활용 가능한 프롬프트 작성 노하우를 쌓는 데 도움이 됩니다.

### 5.1. Azure 아키텍처 설계 조언 받기

In [11]:
architecture_prompt = """당신은 수석 Azure 솔루션 아키텍트입니다. 다음 요구사항에 맞는 Azure 아키텍처를 설계해주세요:

요구사항:
- 고가용성 웹 애플리케이션 (ASP.NET Core)
- 실시간 데이터 처리 (IoT 디바이스에서 수집된 센서 데이터)
- 대용량 데이터 저장 및 분석
- AI 기반 예측 모델을 통한 이상 징후 감지
- 철저한 보안 및 규정 준수 (개인정보 포함)
- 비용 효율적인 설계

다음 사항을 포함해주세요:
1. 아키텍처 다이어그램 (텍스트로 표현)
2. 각 Azure 서비스의 선택 이유
3. 확장성 및 장애 조치 전략
4. 예상 비용 최적화 방안
5. 보안 고려사항"""

print("=== Azure 아키텍처 설계 조언 ===\n")
print(get_completion(architecture_prompt))

=== Azure 아키텍처 설계 조언 ===

아래는 요구사항에 맞춘 Azure 아키텍처 설계입니다.

### 1. 아키텍처 다이어그램 (텍스트로 표현)

```
+-------------------+        +--------------------+        +--------------------+
|   IoT 디바이스    | -----> |   Azure IoT Hub    | -----> |   Azure Stream     |
|                   |        |                    |        |   Analytics         |
+-------------------+        +--------------------+        +--------------------+
                                                                  |
                                                                  v
                                                      +---------------------+
                                                      |   Azure Data Lake   |
                                                      |   Storage Gen2      |
                                                      +---------------------+
                                                                  |
                                                                  v


### 5.2. Azure CLI 명령어 생성

In [12]:
cli_prompt = """Azure CLI 명령어를 사용하여 다음 작업을 수행하는 스크립트를 생성해주세요:

1. 리소스 그룹 생성 (이름: rg-workshop-prod, 위치: Korea Central)
2. Azure Kubernetes Service(AKS) 클러스터 생성 (3개 노드, 시스템 할당 관리 ID 사용)
3. Azure Container Registry(ACR) 생성 및 AKS와 연결
4. Azure Key Vault 생성 및 필요한 비밀 저장 (DB 연결 문자열, API 키)
5. Azure Monitor를 사용하여 AKS 클러스터 모니터링 설정

각 명령어에 대한 간단한 설명을 주석으로 추가해주세요."""

print("=== Azure CLI 명령어 생성 ===\n")
print(get_completion(cli_prompt))

=== Azure CLI 명령어 생성 ===

아래는 Azure CLI를 사용하여 요청하신 작업을 수행하는 스크립트입니다. 각 명령어에 대한 간단한 설명을 주석으로 추가했습니다.

```bash
#!/bin/bash

# 1. 리소스 그룹 생성
az group create --name rg-workshop-prod --location koreaCentral
# 리소스 그룹 이름: rg-workshop-prod, 위치: Korea Central

# 2. Azure Kubernetes Service(AKS) 클러스터 생성
az aks create --resource-group rg-workshop-prod --name aks-workshop-cluster --node-count 3 --enable-managed-identity
# AKS 클러스터 이름: aks-workshop-cluster, 노드 수: 3, 시스템 할당 관리 ID 사용

# 3. Azure Container Registry(ACR) 생성 및 AKS와 연결
az acr create --resource-group rg-workshop-prod --name acrworkshopregistry --sku Basic
# ACR 이름: acrworkshopregistry, SKU: Basic

# ACR와 AKS 클러스터 연결
az aks update --name aks-workshop-cluster --resource-group rg-workshop-prod --attach-acr acrworkshopregistry
# AKS 클러스터에 ACR 연결

# 4. Azure Key Vault 생성 및 필요한 비밀 저장
az keyvault create --name kv-workshop --resource-group rg-workshop-prod --location koreaCentral
# Key Vault 이름: kv-workshop, 리소스 그룹: rg-workshop-prod

# 비밀 저장 (DB 연

## 6. 실습: 나만의 프롬프트 작성하기

이제 지금까지 배운 내용을 바탕으로 나만의 프롬프트를 작성해봅시다.

In [13]:
# 여기에 직접 프롬프트를 작성하고 실행해보세요
my_prompt = """
# 여기에 프롬프트를 작성하세요
"""

# 주석을 제거하고 실행하세요
# print(get_completion(my_prompt))

## 7. 마무리 및 다음 단계

이 세션에서는 LLM과 프롬프트 엔지니어링의 기본 및 고급 기법을 학습했습니다. 다음 세션에서는 GitHub Copilot과 AutoGen을 활용하여 Multi AI Agent 앱을 구현해볼 예정입니다.

### 학습한 내용 요약
- Azure AI Foundry에서 LLM 모델 배포 및 사용 방법
- 기본 프롬프트 엔지니어링 기법 (명확한 지시사항, 역할 부여, 단계별 사고 유도, Few-Shot 학습)
- 고급 프롬프트 엔지니어링 기법 (시스템 메시지, 다중 단계 지시사항, 출력 형식 지정)
- 실전 프롬프트 엔지니어링 사례

### 추가 학습 자료
- [Azure OpenAI Service 공식 문서](https://docs.microsoft.com/azure/cognitive-services/openai/)
- [Azure AI Foundry 공식 문서](https://docs.microsoft.com/azure/ai-services/)
- [Generative AI for Beginners (Microsoft)](https://github.com/microsoft/generative-ai-for-beginners)

### LLM 파라미터: temperature와 top-p

- **temperature**: 생성되는 텍스트의 다양성과 창의성을 조절하는 값입니다. 0에 가까울수록 더 결정적이고 일관된 답변을, 1에 가까울수록 더 다양하고 창의적인 답변을 생성합니다. 일반적으로 0.2~0.8 사이의 값을 많이 사용합니다.
- **top-p (nucleus sampling)**: 모델이 다음 단어를 선택할 때 누적 확률이 top-p 이하가 되도록 후보군을 제한합니다. 1에 가까울수록 다양한 답변이, 0.5 등 낮은 값일수록 더 보수적인 답변이 생성됩니다.
- 두 파라미터를 조합해 답변의 창의성, 다양성, 일관성을 실험할 수 있습니다.

아래 코드에서 temperature와 top_p 값을 변경해 다양한 응답을 실험해보세요.

In [14]:
# temperature와 top_p를 조정하여 응답의 다양성 실험 (동기적으로 처리)
prompt = "Azure OpenAI의 주요 특징을 한 문장으로 설명해줘."

response1 = client.chat.completions.create(
    model=deployment_name,
    messages=[{"role": "user", "content": prompt}],
    temperature=0.1,
    top_p=0.5,
    max_tokens=50
)
print("temperature=0.2, top_p=0.5:")
print(response1.choices[0].message.content)

response2 = client.chat.completions.create(
    model=deployment_name,
    messages=[{"role": "user", "content": prompt}],
    temperature=0.9,
    top_p=1.0,
    max_tokens=50
)
print("\ntemperature=0.8, top_p=1.0:")
print(response2.choices[0].message.content)

temperature=0.2, top_p=0.5:
Azure OpenAI는 Microsoft Azure 플랫폼에서 제공되는 강력한 AI 모델을 활용하여 자연어 처리, 이미지 생성 등 다양한 AI 기능을 클라우드 환경에서 안전하고 확장 가능하게 사용할 수 있는 서비스입니다.

temperature=0.8, top_p=1.0:
Azure OpenAI는 Microsoft Azure 플랫폼에서 강력한 AI 모델을 제공하여 기업이 자연어 처리, 코드 생성, 데이터 분석 등의 다양한 AI 기능을 손쉽게 통합하고 활용할 수 있도록 지원하는 서비스입니다.
